# Data Preprocessing for Machine Learning

Before feeding data into a machine learning model, it's crucial to preprocess the data to ensure it's in a usable format. Preprocessing can include:

1. Handling missing values
2. Handling non-numerical values

## 1. Handling Missing Values

Missing data can distort the performance of machine learning models. There are several strategies to handle missing values:

- **Imputation**: Replace missing values with a statistic like the mean, median, or mode of the column.
- **Deletion**: Remove records with missing values, which is feasible if the number of these records is insignificant.

> **Note**: The choice of strategy depends on the context and proportion of missing data. 

Below we'll import a dataset and demonstrate how to identify and handle missing values.

In [1]:
### Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load Dataframe
car_sales_df = pd.read_csv("data/car-sales.csv")
car_sales_df.head(10)

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
5,Honda,Red,42652.0,4.0,23883.0
6,Toyota,Blue,163453.0,4.0,8473.0
7,Honda,White,NaN,4.0,20306.0
8,NaN,White,130538.0,4.0,9374.0
9,Honda,Blue,51029.0,4.0,26683.0


In [2]:
# Display number of missing values in each column
car_sales_df.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

### 1.1 Fill Missing Data with pandas

To handle missing data effectively, we differentiate between categorical and numerical features. We will fill missing categorical values with a placeholder label "missing" and use statistical methods for numerical features. For sensitive fields like 'Price', which is also our target variable, we will remove rows to avoid creating false labels that could mislead our model.

> **Note**: Imputation is a common practice for dealing with missing data, but there's no one-size-fits-all solution. The method you choose should be informed by the nature of your dataset and the specific characteristics of the missing data. For more advanced techniques, research "data imputation techniques".

In [3]:
# Fill missing values for categorical features with the placeholder 'missing'
car_sales_df["Make"].fillna("missing", inplace=True)
car_sales_df["Colour"].fillna("missing", inplace=True)

# For the numerical feature 'Odometer (KM)', we calculate the mean and use it to fill missing values
# Ensure there are non-missing values to calculate the mean, otherwise, skip this step
car_sales_df["Odometer (KM)"].fillna(car_sales_df["Odometer (KM)"].mean(), inplace=True)

# Assuming '4' is the mode or most common value for the 'Doors' feature
car_sales_df["Doors"].fillna(4, inplace=True)

# Display the number of missing values for each column to verify no missing data remains
print(car_sales_df.isna().sum())

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64


In [4]:
# Remove rows where the target variable 'Price' is missing as imputation might introduce bias
car_sales_df.dropna(subset=['Price'], inplace=True)

# Check again for missing values to confirm 'Price' rows are removed
print(car_sales_df.isna().sum())

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64


### 1.2 Fill Missing Data with scikit-learn's SimpleImputer

Scikit-learn's `SimpleImputer` class provides a convenient way to handle missing values using various strategies. For categorical data, we can fill in missing values with a placeholder, such as 'missing', and for numerical data, we can replace missing values with the mean of the available values.

First we will need to reimport the data and check for missing values

In [5]:
# Reimport the DataFrame
car_sales_df = pd.read_csv("data/car-sales.csv")
car_sales_df.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

We'll remove rows with missing target values ('Price') to avoid introducing inaccuracies into our model:

In [6]:
# Drop the rows with missing in the "Price" column
car_sales_df.dropna(subset=["Price"], inplace=True)
car_sales_df.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

It is crucial to split the data into training and test sets before any imputation. This is to prevent data leakage from the test set into the training set, which can lead to overly optimistic performance estimates. We learn the imputation parameters (e.g., the mean of a column) from the training set to avoid any influence from the test set.

In [7]:
# Split the data into features and target
X = car_sales_df.drop("Price", axis=1)
y = car_sales_df["Price"]

# Split the data into training and test sets to prevent data leakage during imputation
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Imputation strategies
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")
door_imputer = SimpleImputer(strategy="constant", fill_value=4)
num_imputer = SimpleImputer(strategy="mean")

# Columns to apply imputers
categorical_features = ["Make", "Colour"]
door_feature = ["Doors"]
numerical_feature = ["Odometer (KM)"]

# Column transformer
imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, categorical_features),
    ("door_imputer", door_imputer, door_feature),
    ("num_imputer", num_imputer, numerical_feature)
])

We apply fit_transform() to the training data to learn the imputation parameters, and then we use transform() on the test data to apply these parameters without learning from the test data.

In [9]:
# Fit the imputers on the training set and transform both the training and test sets
filled_X_train = imputer.fit_transform(X_train)
filled_X_test = imputer.transform(X_test)

In [10]:
# Verify filled X_train
filled_X_train

array([['Honda', 'White', 4.0, 71934.0],
       ['Toyota', 'Red', 4.0, 162665.0],
       ['Honda', 'White', 4.0, 42844.0],
       ...,
       ['Toyota', 'White', 4.0, 196225.0],
       ['Honda', 'Blue', 4.0, 133117.0],
       ['Honda', 'missing', 4.0, 150582.0]], dtype=object)

In [11]:
# Revert transformed data array's back into DataFrame's
car_sales_filled_train = pd.DataFrame(filled_X_train, columns=X.columns)
car_sales_filled_test = pd.DataFrame(filled_X_test, columns=X.columns)

# Verify that there are no missing values in the training set
car_sales_filled_train.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
dtype: int64

## 2. Handling Non-numerical Values

Non-numerical values present in categorical features need to be converted into a numerical format that machine learning models can interpret. One common approach is OneHotEncoding, which converts categorical variables into a format that can be provided to ML algorithms to do a better job in prediction.

First, we reload our dataset and check for any missing values that need to be addressed before proceeding with encoding. We then split our data into features (`X`) and target (`y`) and further into training and test sets. This is a crucial step to prevent data leakage during imputation and ensures that the model is evaluated on data it has never seen before, thus giving us a better estimate of its performance on new data.

In [12]:
# Reimport the DataFrame
car_sales_df = pd.read_csv("data/car-sales.csv")
car_sales_df.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [13]:
car_sales_df.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

In [14]:
# Split the data into features and target
X = car_sales_df.drop("Price", axis=1)
y = car_sales_df["Price"]

# Split the data into training and test sets to prevent data leakage during imputation
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### 2.1 Handling Non-numerical Values with Scikit-learn

Scikit-learn's `ColumnTransformer` and `OneHotEncoder` are robust tools for handling categorical data. We'll apply these to our categorical features to ensure they're model-ready.

In [15]:
X.head()

,Make,Colour,Odometer (KM),Doors
0,Honda,White,35431.0,4.0
1,BMW,Blue,192714.0,5.0
2,Honda,White,84714.0,4.0
3,Toyota,White,154365.0,4.0
4,Nissan,Blue,181577.0,3.0


In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Assume X is your input features DataFrame
categorical_features = ["Make", "Colour", "Doors"]  # specify your categorical columns
one_hot_encoder = OneHotEncoder()

# Create the ColumnTransformer with OneHotEncoder for categorical features
transformer = ColumnTransformer(
    transformers=[
        ("one_hot", one_hot_encoder, categorical_features)
    ],
    remainder="passthrough"  # other columns that are not listed will be passed through without transformation
)

# Apply the ColumnTransformer to your data
transformed_X = transformer.fit_transform(X)

In [17]:
# Convert the transformed_X to a dense format
transformed_X_dense = transformed_X.toarray()

# Convert the result to a DataFrame
# Will require new column names from the one_hot_encoder. get_feature_names_out method can give you the new column names for the one-hot encoded columns
new_column_names = transformer.named_transformers_['one_hot'].get_feature_names_out(input_features=categorical_features)

# The remainder of the columns will be the non-categorical features, which we assume are passed through
# We concatenate them with the new column names from the one-hot encoder
all_column_names = list(new_column_names) + [col for col in X.columns if col not in categorical_features]

# Finally, create the DataFrame with the new column names
transformed_X_df = pd.DataFrame(transformed_X_dense, columns=all_column_names)

# Show the resulting DataFrame
transformed_X_df.head()

,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Make_nan,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White,Colour_nan,Doors_3.0,Doors_4.0,Doors_5.0,Doors_nan,Odometer (KM)
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,181577.0


### 2.2 Handling Non-numerical Values with pandas
Pandas provides a convenient function called `get_dummies()` which can apply one-hot encoding to our dataset. This function converts categorical variable(s) into dummy/indicator variables that are either `0` or `1`. For each unique value in a categorical column, `get_dummies()` creates a new column that indicates the presence of the value with `1` and `0` otherwise.

The resulting DataFrame will have additional columns - one for each unique value in the categorical columns we've selected to encode.

Let's apply this to our 'Make', 'Colour', and 'Doors' columns.

In [18]:
dummies = pd.get_dummies(car_sales_df[["Make", "Colour", "Doors"]], dtype=int)
dummies.head()

,Doors,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White
0,4.0,0,1,0,0,0,0,0,0,1
1,5.0,1,0,0,0,0,1,0,0,0
2,4.0,0,1,0,0,0,0,0,0,1
3,4.0,0,0,0,1,0,0,0,0,1
4,3.0,0,0,1,0,0,1,0,0,0
